# 3주차 : 단락 연결 여부 확인

In [390]:
#!pip install transformers

In [391]:
from transformers import BertForMaskedLM, FillMaskPipeline
from transformers import TFBertForNextSentencePrediction, BertForNextSentencePrediction, AutoTokenizer, BertTokenizer
from transformers import BertConfig, AdamW
import tensorflow as tf
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn.functional import softmax
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.callbacks import LearningRateMonitor
from tqdm import tqdm
from typing import Callable, Tuple
from seqeval.metrics import accuracy_score
import json
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import os
import random
import MeCab
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

In [392]:
#device = torch.device("cuda:0")   #gpu 사용
device = torch.device("cpu")

print('GPUs Available :', torch.cuda.device_count())

if torch.cuda.is_available():
    print('GPU running')
else:
    print('GPU not running')

GPUs Available : 1
GPU running


## 1. 문장 연결성 파악

2개 단락의 연결되는 문장이 자연스럽게 연결되는지 확인하는 것이 목적이다.  
이를 위해 KLUE(Korean Lnaguage Understanding Evaluation)를 통해 배포된 BERT 모델을 이용할 것이다. 

####  KLUE는 총 8가지 자연어이해(NLU) task로 구성되어 있다.  

    -Topic Classification  
    -Semantic Textual  
    -Natural Language Inference  
    -Named Entity Recognition  
    -Relation Extraction  
    -Dependency parsing  
    -Machine Reading Comprehension  
    -Dialogue State Tracking  

이 중에서 여기서 진행할 것은 Natural Language Inference(자연어추론)이다.  


## 방법: MASK에 들어갈 단어 예측
    -TFBertForMaskedLM 모델 이용
    2개 단락이 연결되는 부분을 마스킹해 모델에 넣었을 때,
    실제 정답이 결과값 5개 안에 있다면, 연결된 문장은 자연스럽다고 볼 수 있을 것이라 가정

In [393]:
model = BertForMaskedLM.from_pretrained('klue/bert-base')  
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")   #해당 모델이 학습되었을 당시 사용된 토크나이저

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [394]:
inputs = tokenizer('축구는 정말 재미있는 [MASK]다.', return_tensors='tf')  
print(inputs['input_ids'])
print(inputs['token_type_ids'])   #문장 길이만큼의 0 시퀀스
print(inputs['attention_mask'])   #실제 단어와 패딩 토큰을 구분하기 위한 어텐션 마스크

tf.Tensor([[   2 4713 2259 3944 6001 2259    4  809   18    3]], shape=(1, 10), dtype=int32)
tf.Tensor([[0 0 0 0 0 0 0 0 0 0]], shape=(1, 10), dtype=int32)
tf.Tensor([[1 1 1 1 1 1 1 1 1 1]], shape=(1, 10), dtype=int32)


In [395]:
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)   #[MASK] 위치에 들어갈 상위 5개 출력
pip('축구는 정말 재미있는 [MASK]다.')

[{'score': 0.896362841129303,
  'token': 4559,
  'token_str': '스포츠',
  'sequence': '축구는 정말 재미있는 스포츠 다.'},
 {'score': 0.025958051905035973,
  'token': 568,
  'token_str': '거',
  'sequence': '축구는 정말 재미있는 거 다.'},
 {'score': 0.010034135542809963,
  'token': 3682,
  'token_str': '경기',
  'sequence': '축구는 정말 재미있는 경기 다.'},
 {'score': 0.007924516685307026,
  'token': 4713,
  'token_str': '축구',
  'sequence': '축구는 정말 재미있는 축구 다.'},
 {'score': 0.007844332605600357,
  'token': 5845,
  'token_str': '놀이',
  'sequence': '축구는 정말 재미있는 놀이 다.'}]

In [396]:
#원래 단어는 '지지자'
pip('더불어민주당 [MASK] 중 39.5%, 국민의당 지지자 중 28.8%, 기타 21%, 무당층의 16.2%, 개혁보수당 지지자 중 12.3%였다.')   

[{'score': 0.9840489625930786,
  'token': 11006,
  'token_str': '지지자',
  'sequence': '더불어민주당 지지자 중 39. 5 %, 국민의당 지지자 중 28. 8 %, 기타 21 %, 무당층의 16. 2 %, 개혁보수당 지지자 중 12. 3 % 였다.'},
 {'score': 0.01445611473172903,
  'token': 13755,
  'token_str': '지지층',
  'sequence': '더불어민주당 지지층 중 39. 5 %, 국민의당 지지자 중 28. 8 %, 기타 21 %, 무당층의 16. 2 %, 개혁보수당 지지자 중 12. 3 % 였다.'},
 {'score': 0.0006135448929853737,
  'token': 9440,
  'token_str': '응답자',
  'sequence': '더불어민주당 응답자 중 39. 5 %, 국민의당 지지자 중 28. 8 %, 기타 21 %, 무당층의 16. 2 %, 개혁보수당 지지자 중 12. 3 % 였다.'},
 {'score': 0.00022629096929449588,
  'token': 4315,
  'token_str': '지지',
  'sequence': '더불어민주당 지지 중 39. 5 %, 국민의당 지지자 중 28. 8 %, 기타 21 %, 무당층의 16. 2 %, 개혁보수당 지지자 중 12. 3 % 였다.'},
 {'score': 0.00022148765856400132,
  'token': 7243,
  'token_str': '유권자',
  'sequence': '더불어민주당 유권자 중 39. 5 %, 국민의당 지지자 중 28. 8 %, 기타 21 %, 무당층의 16. 2 %, 개혁보수당 지지자 중 12. 3 % 였다.'}]

In [397]:
#원래 단어는 '이런'
pip('모른다고 답하거나 응답하지 않은 비율은 1.5%였다. [MASK] 결과를 우리나라 인구수 5168만여명에 대입하면 촛불집회 참가 경험이 있는 국민은 1199만여명으로 나온다.')

[{'score': 0.4670316278934479,
  'token': 1504,
  'token_str': '이',
  'sequence': '모른다고 답하거나 응답하지 않은 비율은 1. 5 % 였다. 이 결과를 우리나라 인구수 5168만여명에 대입하면 촛불집회 참가 경험이 있는 국민은 1199만여명으로 나온다.'},
 {'score': 0.20466183125972748,
  'token': 3742,
  'token_str': '조사',
  'sequence': '모른다고 답하거나 응답하지 않은 비율은 1. 5 % 였다. 조사 결과를 우리나라 인구수 5168만여명에 대입하면 촛불집회 참가 경험이 있는 국민은 1199만여명으로 나온다.'},
 {'score': 0.1495237499475479,
  'token': 3686,
  'token_str': '이번',
  'sequence': '모른다고 답하거나 응답하지 않은 비율은 1. 5 % 였다. 이번 결과를 우리나라 인구수 5168만여명에 대입하면 촛불집회 참가 경험이 있는 국민은 1199만여명으로 나온다.'},
 {'score': 0.08464939147233963,
  'token': 3667,
  'token_str': '이런',
  'sequence': '모른다고 답하거나 응답하지 않은 비율은 1. 5 % 였다. 이런 결과를 우리나라 인구수 5168만여명에 대입하면 촛불집회 참가 경험이 있는 국민은 1199만여명으로 나온다.'},
 {'score': 0.03338053822517395,
  'token': 7509,
  'token_str': '설문',
  'sequence': '모른다고 답하거나 응답하지 않은 비율은 1. 5 % 였다. 설문 결과를 우리나라 인구수 5168만여명에 대입하면 촛불집회 참가 경험이 있는 국민은 1199만여명으로 나온다.'}]

## 2. 단락 유사도 파악 및 동일 문장 여부 파악
    2개 단락의 유사도 또는 연결성이 있는지,
    2개의 단락에서 문장이 끊어졌을 때 1개의 문장으로 붙여지는지 확인하는 것이 목적이다.

## 방법: 다음 문장을 예측
    -BertForNextSentencePrediction 모델 이용
    실제로 전후관계인 단락(0:True), 가짜로 묶어놓은 단락(1:False)을 학습시킨다.
    이후 모델에 테스트할 앞뒤 단락을 넣었을 때, True/False를 잘 판별해낸다면,
    두 단락의 유사도/연결성이 있는지, 나아가 한 문장으로 잘 붙는지 확인할 수 있을 것이라 가정

    https://towardsdatascience.com/how-to-use-bert-from-the-hugging-face-transformer-library-d373a22b0209

In [398]:
model = TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForNextSentencePrediction: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


In [399]:
# XML파일에서 추출한 이어지는 두 개의 문장
prompt = '이런 결과를 지난해 11월 행정자치부가 발표한 우리나라 인구수 5168만여명에 대입하면 ' 
next_sentence = ' 촛불집회 참가 경험이 있는 국민은 1199만여명으로 나온다.'
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [0]


In [400]:
# XML파일에서 추출한 이어지는 제목, 문장
prompt = '서브프라임 모기지 사태로 촉발된 미국의 금융위기가 미국 금융기관을' 
next_sentence = ' 몰락시키고, 세계 굴지의 자동차 회사들을 생존위기로 내몬 데 이어 최근 언론사까지 벼랑 끝으로 내몰고 있다.'
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [0]


In [401]:
# XML파일에서 추출한 상관없는 두 개의 문장
prompt = '75.3%는 전혀 참여한 적이 없다고 응답했고, 모른다고 답하거나 응답하지 않은 비율은 1.5%였다.' 
next_sentence = '이 목록에 오른 493개 차량 모델 중 LG화학과 삼성SDI의 배터리를 탑재한 차량은 하나도 없었다.'
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [1]


In [402]:
# XML파일에서 추출한 상관없는 제목, 문장
prompt = '남의 집 제집이라 속여 팔았는데 사기 아니라고?' 
next_sentence = '충남 금산군이 환경부가 수변구역으로 지정한 임야에 버섯재배사 개발허가를 내줘 빈축을 사고 있다.'
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [1]


### => TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt=True) 에
### 모두의말뭉치 Corpus를 이어서 학습시킬 예정

## 데이터 준비

In [403]:
#모두의말뭉치 파일 내 json 파일 목록
def parse_paths(folder):
    for current, dirs, files in os.walk(folder):
        return [os.path.join(current, file) for file in files if file.endswith(".json")]

#json 파일 내 기사 내용 추출
def parse_contents(files):
    result = []
    for file in tqdm(files, desc="[Contents Parsing]"):    #진행상황 확인용
        with open(file, "r", encoding="utf-8") as f:
            contents = json.load(f)
            for doc in contents['document']:
                result_paragraph = []     #같은 기사 한문단
                paragraph_id = doc['id']  #같은 기사 id
                for paragraph in doc['paragraph']:
                    if paragraph['id'][-2:] == '.1':   #기사의 제목 부분은 제외
                        continue
                    elif paragraph['id'][:17]==paragraph_id[:17] and paragraph['id'][-2:]=='.2':  #같은 id(같은 단락의 기사)의 첫줄이라면
                        continue
                    elif paragraph['id'][:17]==paragraph_id[:17] and paragraph['id'][-2:]!='.2':  #같은 id(같은 단락의 기사)의 첫줄이 아닌 문장
                        result_paragraph.append(paragraph['form'])
                    else:   #다른 id(다른 단락의 기사)라면
                        result.append(result_paragraph)   #지금까지 id의 문장들은 result로
                        result_pragraph = []   #result_paragraph 비우기
                        paragraph_id = paragraph['id'][:17]
                        result_paragraph.append(paragraph['form'])
    print("[Contents Length] {0:,}".format(len(result)))
    
    for i in range(len(result)):
        pa = ''
        for sentence in result[i]:
            pa += f' {sentence}'
            result[i] = pa    
    return result
data_path = 'C:\BootCamp\CP2\Korpus'
data = parse_paths(data_path)
contents = parse_contents(data)

[Contents Parsing]: 100%|██████████| 35/35 [00:19<00:00,  1.80it/s]

[Contents Length] 314


In [404]:
#같은 기사는 한 문단으로 처리됨
contents[0]

' 전남 목포시는 최근 이목이 집중되고 있는 근대역사문화공간 재생활성화 사업을 근대문화재 보존과 활용이라는 당초 취지대로 차질없이 추진하겠다고 밝혔다. 이 사업은 목포 원도심인 유달‧만호동 일대에 산재해 있는 근대건축물 등 문화유산을 보존하고 보수‧정비하는 사업으로 금년부터 향후 5년간 총 사업비 500억원이 투입된다. 목포시는 원도심 일대의 근대경관을 회복하고 거주민 생활여건 개선과 관광인프라 확충 등을 통해 이 지역을 전국적인 근대 문화유산의 보고로 만들어 나간다는 계획이다. 아울러, 최근 언론의 집중 보도로 목포 근대문화재에 대해 관심이 많아진 이 기회를 문화유산 보존의 필요성과 당위성을 널리 알리고 이에 대한 사회적 합의를 이루는 계기로 만드는 노력도 기울이기로 했다. 올 해는 종합정비계획을 수립하고 역사문화공간 내 건축자산 매입 및 정비에 나선다. 개별문화재로 등록된 15개소를 중심으로 우선 매입하고, 역사적‧건축적 가치가 높은 건축물과 경관을 훼손하는 건축물도 매입해서 공공재로의 활용을 확대한다는 계획이다. 특히, 목포시는 건축자산 매입 시 공정하고 투명한 행정절차를 통해 투기자본 유입을 원천 차단한다. 또, 근대역사문화공간 내 보존 활용, 관리 및 지원 기준에 관한 조례를 제정해 젠트리피케이션 발생을 예방하고, 특정 투기세력들이 수혜를 받을 수 없도록 제도적 장치를 마련하기로 했다. 목포시는 근대역사문화공간 재생 활성화 시범사업이 목포의 역사적 가치를 보존하면서 지역발전도 함께 이룰 수 있는 중요한 기회이자, 소중한 문화유산을 지키기 위해 꼭 필요한 사업이라는 점을 강조했다. 아울러, 문화재청, 관련 기관 등과 잘 협력해 사업을 본래의 취지대로 흔들림없이 추진해서 반드시 근대문화재 보존활용의 성공 모델로 만들어 간다는 방침을 확고히 했다.'

In [405]:
len_paragraph = []
for i in contents:
    len_paragraph.append(len(i))
    
print(f'기사의 길이 : 최소 {min(len_paragraph)}자, 최대 길이 {max(len_paragraph)}자')

#앞문단 128자 + 뒷문단 128자 = 256자를 pre-trained 모델에 fine-tuning하기 위한 입력값으로 사용
#예를들어 최대길이인 4135자를 입력값으로 쓴다면, 4135 // 128 = 32개의 문단으로 나누고,
#앞-뒤관계가 True인 총 30쌍의 데이터를 얻을 수 있다.

기사의 길이 : 최소 406자, 최대 길이 4135자


## 데이터 준비

    -기사 총 314개를 각각 128자씩 분할

In [406]:
def paragraph_slicing(data, max_len):
    result_data = []
    for paragraph in data:
        result_paragraph = []
        for i in range(len(paragraph)//max_len +1):
            start_idx = max_len * i
            end_idx = start_idx + max_len
            result_paragraph.append(paragraph[start_idx:end_idx])   #[128자 text]
        result_data.append(result_paragraph)
    return result_data

news = paragraph_slicing(contents, 128)

In [407]:
total_128 = 0
for i in news:
    total_128 += len(i)
    
print('원래 문단 개수 :', len(news))
print('128자로 나눈 후 개수 :', total_128)
news[0]

원래 문단 개수 : 314
128자로 나눈 후 개수 : 2947


[' 전남 목포시는 최근 이목이 집중되고 있는 근대역사문화공간 재생활성화 사업을 근대문화재 보존과 활용이라는 당초 취지대로 차질없이 추진하겠다고 밝혔다. 이 사업은 목포 원도심인 유달‧만호동 일대에 산재해 있는 근대건축물 등 문화유산',
 '을 보존하고 보수‧정비하는 사업으로 금년부터 향후 5년간 총 사업비 500억원이 투입된다. 목포시는 원도심 일대의 근대경관을 회복하고 거주민 생활여건 개선과 관광인프라 확충 등을 통해 이 지역을 전국적인 근대 문화유산의 보고로 만',
 '들어 나간다는 계획이다. 아울러, 최근 언론의 집중 보도로 목포 근대문화재에 대해 관심이 많아진 이 기회를 문화유산 보존의 필요성과 당위성을 널리 알리고 이에 대한 사회적 합의를 이루는 계기로 만드는 노력도 기울이기로 했다. 올 ',
 '해는 종합정비계획을 수립하고 역사문화공간 내 건축자산 매입 및 정비에 나선다. 개별문화재로 등록된 15개소를 중심으로 우선 매입하고, 역사적‧건축적 가치가 높은 건축물과 경관을 훼손하는 건축물도 매입해서 공공재로의 활용을 확대한다',
 '는 계획이다. 특히, 목포시는 건축자산 매입 시 공정하고 투명한 행정절차를 통해 투기자본 유입을 원천 차단한다. 또, 근대역사문화공간 내 보존 활용, 관리 및 지원 기준에 관한 조례를 제정해 젠트리피케이션 발생을 예방하고, 특정 ',
 '투기세력들이 수혜를 받을 수 없도록 제도적 장치를 마련하기로 했다. 목포시는 근대역사문화공간 재생 활성화 시범사업이 목포의 역사적 가치를 보존하면서 지역발전도 함께 이룰 수 있는 중요한 기회이자, 소중한 문화유산을 지키기 위해 꼭',
 ' 필요한 사업이라는 점을 강조했다. 아울러, 문화재청, 관련 기관 등과 잘 협력해 사업을 본래의 취지대로 흔들림없이 추진해서 반드시 근대문화재 보존활용의 성공 모델로 만들어 간다는 방침을 확고히 했다.']

## Input data 만들기

위에서 TFBertForNextSentencePrediction 모델을 사용했던 형식을 보면

    prompt = '앞문장' 
    next_sentence = '뒷문장'
    encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

    logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

    softmax = tf.keras.layers.Softmax()
    probs = softmax(logits)
    print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

=>예를들어, 문단이 128자로 나눴을 때 총 5개로 나눠졌다면, 각 문단의 idx 0-1, 1-2, 2-3, 3-4(총 4번) 문장별로 입력되어야 한다.

In [408]:
i = 0  
tf_next_sentence = []
true, false = [], []

for i in range(len(news)):    #314개 기사를 하나씩 확인
    content = news[i]    #하나의 기사
    for j in range(len(content)):    #각 기사 내 문단을 하나씩 확인
        for j in range(len(content)-1):   
            true.append((content[j], content[j+1], 0))   #(앞문단, 진짜뒷문단, 0(True))
            
            while 0:
                random_news = random.randint(0, len(news))   #314개 중 무작위 기사를 뽑아서
                random_content = random.randint(0, len(news[random_news]))   #무작위로 한 문단 가져오기
                try:
                    false_text = news[random_news][random_content]
                    break
                except:
                    continue
            
            false.append((content[j], false_text, 1))   #(앞문단, 가짜뒷문단, 1(False))
            tf_next_sentence.extend(true)
            tf_next_sentence.extend(false)
            true, false = [], []
        j += 1
    i += 1

print('기존 기사 개수 :', len(news))
print('T/F 처리 후 데이터 개수 :', len(tf_next_sentence))

기존 기사 개수 : 314
T/F 처리 후 데이터 개수 : 64824


In [424]:
tf_next_sentence[:6]

[(' 전남 목포시는 최근 이목이 집중되고 있는 근대역사문화공간 재생활성화 사업을 근대문화재 보존과 활용이라는 당초 취지대로 차질없이 추진하겠다고 밝혔다. 이 사업은 목포 원도심인 유달‧만호동 일대에 산재해 있는 근대건축물 등 문화유산',
  '을 보존하고 보수‧정비하는 사업으로 금년부터 향후 5년간 총 사업비 500억원이 투입된다. 목포시는 원도심 일대의 근대경관을 회복하고 거주민 생활여건 개선과 관광인프라 확충 등을 통해 이 지역을 전국적인 근대 문화유산의 보고로 만',
  0),
 (' 전남 목포시는 최근 이목이 집중되고 있는 근대역사문화공간 재생활성화 사업을 근대문화재 보존과 활용이라는 당초 취지대로 차질없이 추진하겠다고 밝혔다. 이 사업은 목포 원도심인 유달‧만호동 일대에 산재해 있는 근대건축물 등 문화유산',
  "속 소통혁신정책관에게 부여한 '도민 갈등 관리'기능이 제대로 작동하지 않았던 점 역시 원 도정이 서둘러 해결해야 할 과제로 꼽힌다. 이달 중 수립할 예정인 갈등관리 종합계획 역시 예방적 갈등관리와 현안 공공갈등 조정 및 해소라는 ",
  1),
 ('을 보존하고 보수‧정비하는 사업으로 금년부터 향후 5년간 총 사업비 500억원이 투입된다. 목포시는 원도심 일대의 근대경관을 회복하고 거주민 생활여건 개선과 관광인프라 확충 등을 통해 이 지역을 전국적인 근대 문화유산의 보고로 만',
  '들어 나간다는 계획이다. 아울러, 최근 언론의 집중 보도로 목포 근대문화재에 대해 관심이 많아진 이 기회를 문화유산 보존의 필요성과 당위성을 널리 알리고 이에 대한 사회적 합의를 이루는 계기로 만드는 노력도 기울이기로 했다. 올 ',
  0),
 ('을 보존하고 보수‧정비하는 사업으로 금년부터 향후 5년간 총 사업비 500억원이 투입된다. 목포시는 원도심 일대의 근대경관을 회복하고 거주민 생활여건 개선과 관광인프라 확충 등을 통해 이 지역을 전국적인 근대 문화유산의 보고로 만',
  "속 소통혁신정책관에게 부여한 '도민 갈등 관리'기능이 제대로 작동하지 않았

In [410]:
train, test = train_test_split(tf_next_sentence, test_size=0.2, random_state=0, shuffle=True)
train, val = train_test_split(train, test_size=0.2, random_state=0, shuffle=True)
print("train :", len(train), "  val :", len(val), "  test :", len(test))

train : 41487   val : 10372   test : 12965


## 모델 생성
    CorpusDataset
    Preprocessor
    BERTNextSentenceModel
    Config

In [426]:
class CorpusDataset(Dataset):
    def __init__(self, sentences, transform: Callable):
        self.sentences = sentences
        self.transform = transform

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        prompt = self.sentences[idx][0]
        next_sentence = self.sentences[idx][1]
        label = self.sentences[idx][2]
        (
            input_ids,
            attention_mask,
            token_type_ids,
            label, 
        ) = self.transform(prompt, next_sentence, label)

        return input_ids, attention_mask, token_type_ids, label

In [448]:
class Preprocessor :
    def __init__(self, max_len: int):
        self.tokenizer = BertTokenizer.from_pretrained("snunlp/KR-Medium", do_lower_case=False)
        self.max_len = max_len
        self.pad_token_id = 0

    def get_input_features(self, prompt, next_sentence, label
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        """두 문장(prompt, next_sentence)에 대해 tokenize한 뒤 id로 변환, 두 문장을 이어붙인다.

        Args:
            prompt: 이전 문장
            next_sentence: 이어질 다음 문장
            label : 두 문장이 이어지면 0, 아니면 1

        Returns:
            feature를 리턴한다.
            input_ids : 각 토큰의 id, 2(CLS)로 시작, 3(SEP)
            attention_mask : padding은 0, 데이터가 존재하면 1
            token_type_ids : prompt 위치는 0, next_sentence 위치는 1, 1 뒤의 0은 패딩
            label : 정답이면 [1, 0], 오답이면 [0, 1]
        """
        tok_prompt = self.tokenizer.tokenize(prompt)
        tok_next_sen = self.tokenizer.tokenize(next_sentence)

        input_ids = [2] + self.tokenizer.convert_tokens_to_ids(tok_prompt) + [3]    # 처음과 끝 표시
        input_ids_next_sen = [2]+ self.tokenizer.convert_tokens_to_ids(tok_next_sen)+ [3]

        slicing_idx = 0
        if len(input_ids) + len(input_ids_next_sen) > self.max_len :
          slicing_idx = len(input_ids) + len(input_ids_next_sen) - self.max_len // 2 + 2
          input_ids = [2] + input_ids[slicing_idx:]
          input_ids_next_sen = input_ids_next_sen[:-slicing_idx] + [3]

        token_type_ids = [0]*len(input_ids)
        token_type_ids.extend([1]*len(input_ids_next_sen))
        input_ids.extend(input_ids_next_sen)
        attention_mask = [1] * len(token_type_ids)
        pad_length = self.max_len-len(input_ids)

        input_ids.extend([0] * pad_length)
        token_type_ids.extend([0] * pad_length) # pad : 0
        attention_mask.extend([0] * pad_length) # pad : 0

        input_ids = torch.tensor(input_ids, dtype=torch.int)
        attention_mask = torch.tensor(attention_mask, dtype=torch.int)
        token_type_ids = torch.tensor(token_type_ids, dtype=torch.int)

        label = [1.0, 0.] if label == 0 else [0., 1.0]
        label = torch.tensor(label, dtype=torch.float)
        return input_ids, attention_mask, token_type_ids, label

In [458]:
class BertOnlyNSPHead(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.seq_relationship = nn.Linear(config.hidden_size, 2)

    def forward(self, pooled_output):
        seq_relationship_score = self.seq_relationship(pooled_output)
        print(seq_relationship_score, len(seq_relationship_score))
        return seq_relationship_score

# NextSentencePredictorOutput
class BERTNextSentenceModel(pl.LightningModule):
    def __init__(self, config, dataset):
        super().__init__()
        print("init")
        self.config = config
        self.dataset = dataset
        self.labels_type = [0,1]
        self.pad_token_id = 0
        self.softmax = torch.nn.Softmax()
        self.bert_config = BertConfig.from_pretrained(
            self.config.bert_model, num_labels=2
        )
        self.model = BertForNextSentencePrediction.from_pretrained(
            self.config.bert_model, config=self.bert_config
        )
        self.cls = BertOnlyNSPHead(config)
        self.dropout = nn.Dropout(self.config.dropout_rate)
        self.linear = nn.Linear(
            self.bert_config.hidden_size, len(self.labels_type)
        )

    def forward(self,
        input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None,
        head_mask=None, inputs_embeds=None, labels=None, output_attentions=None, output_hidden_states=None,
        return_dict=None, **kwargs, ):
        """
        return NextSentencePredictorOutput(
            loss=next_sentence_loss,
            logits=seq_relationship_scores,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

        """
        logits = self.model(input_ids, token_type_ids=token_type_ids).logits
        probs = self.softmax(logits)
        return probs

    def training_step(self, batch, batch_nb):
        input_ids, attention_mask, token_type_ids, label_ids = batch
        
        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=label_ids,
        )

        loss = self._calculate_loss(outputs, label_ids) # slot_labels : labels
        # f1 = self._f1_score(outputs, label_ids)
        acc = self._calculate_accuracy(outputs, label_ids)
        tensorboard_logs = {'train_loss': loss, 'train_acc':acc}
        return {"loss": loss, "acc": acc, "log": tensorboard_logs}

    def training_end(self, batch, batch_nb):
        input_ids, attention_mask, token_type_ids, label_ids = batch

        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        loss = self._calculate_loss(outputs, label_ids) # slot_labels : labels
        # f1 = self._f1_score(outputs, slot_label_ids)
        acc = self._calculate_accuracy(outputs, label_ids)

        tensorboard_logs = {'train_loss': loss, 'train_acc':acc}
        print("training_end : ", tensorboard_logs)
        return {"loss": loss, "acc": acc, "log": tensorboard_logs}

    def training_epoch_end(self, outputs):
        super().on_train_epoch_end()
        print("training_epoch_end")
            
        avg_loss = torch.stack([torch.tensor(x['loss']) for x in outputs]).mean()
        # val_f1 = torch.stack([torch.tensor(x['val_f1']) for x in outputs]).mean()
        acc = torch.stack([torch.tensor(x['acc']) for x in outputs]).mean()
        tensorboard_logs = {'loss': avg_loss, 'acc': acc}
        print('training_epoch_end : ', tensorboard_logs)
        self.log("validation_epoch_end : tensorboard_logs ", tensorboard_logs)

    def validation_step(self, batch, batch_nb):
        input_ids, attention_mask, token_type_ids, slot_label_ids = batch
        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        loss = self._calculate_loss(outputs, slot_label_ids)
        # val_f1 = self._f1_score(outputs, slot_label_ids)
        val_acc = self._calculate_accuracy(outputs, slot_label_ids)
        return {"val_loss": loss, 'val_acc':val_acc}

    def validation_epoch_end(self, outputs):
      
        avg_loss = torch.stack([torch.tensor(x['val_loss']) for x in outputs]).mean()
        # val_f1 = torch.stack([torch.tensor(x['val_f1']) for x in outputs]).mean()
        val_acc = torch.stack([torch.tensor(x['val_acc']) for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss, 'val_acc':val_acc}
        print('validation_epoch_end : ', tensorboard_logs)
        self.log('val_acc', val_acc)
        self.log('val_loss', avg_loss)
        return {'val_acc':val_acc, 'val_loss': avg_loss, 'log': tensorboard_logs}
    
    def validation_end(self, outputs):
        print('validation_end : ')

        val_loss = torch.stack([torch.tensor(x["val_loss"]) for x in outputs]).mean()
        val_acc = torch.stack([torch.tensor(x["val_acc"]) for x in outputs]).mean()
        # val_f1 = torch.stack([x["val_f1"] for x in outputs]).mean()
        tensorboard_logs = {
            "val_loss": val_loss,
            "val_acc" : val_acc
        }
        print("validation_end : ", tensorboard_logs)
        return {'val_acc':val_acc, 'val_loss': val_loss, 'log': tensorboard_logs}
    
    def test_step(self, batch, batch_nb):
        input_ids, attention_mask, token_type_ids, slot_label_ids = batch
        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        # f1 = self._f1_score(gt_slot_labels, pred_slot_labels)
        acc = self._calculate_accuracy(outputs, slot_label_ids)
        loss = self._calculate_loss(outputs, slot_label_ids)
        return {"test_loss": loss, "test_acc": acc}

    def test_end(self, outputs):

        # test_f1 = torch.stack([x["test_f1"] for x in outputs]).mean()
        test_loss = torch.stack([torch.tensor(x["test_loss"]) for x in outputs]).mean()
        test_acc = torch.stack([torch.tensor(x["test_acc"]) for x in outputs]).mean()
        self.log("test_loss", test_loss)
        self.log("test_acc", test_acc)
        # self.log("test_f1", test_f1)
        return {"labels" : [x["labels"] for x in outputs], "test_loss": test_loss,  "test_acc": test_acc}
    
    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([torch.tensor(x['test_loss']) for x in outputs]).mean()
        # f1 = torch.stack([x['test_f1'] for x in outputs]).mean()
        acc = torch.stack([torch.tensor(x['test_acc']) for x in outputs]).mean()
        tensorboard_logs = {'test_loss': avg_loss, 'test_acc':acc}
        print('test_epoch_end : ', tensorboard_logs)
        self.log("test_epoch_end : tensorboard_logs ", tensorboard_logs)
        return {'test_acc':acc, 'test_loss': avg_loss, 'log': tensorboard_logs}

    def predict_step(self, batch, batch_idx, dataloader_idx=0):   # prediction : forward(), predict_step()
        input_ids, attention_mask, token_type_ids, label_ids = batch    # slot_label은 없음.
        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        return {'pred_labels':outputs, 'slot_label_ids': label_ids}


    def configure_optimizers(self):
        return AdamW(self.model.parameters(), lr=2e-5, eps=1e-8)

    def train_dataloader(self):
        return DataLoader(self.dataset["train"], batch_size=self.config.eval_batch_size)

    def val_dataloader(self):
        return DataLoader(self.dataset["val"], batch_size=self.config.eval_batch_size)

    def test_dataloader(self):
        return DataLoader(self.dataset["test"], batch_size=self.config.eval_batch_size)

    def pred_dataloader(self, dataset):
        return DataLoader(dataset, batch_size=1)

    def _calculate_loss(self, outputs, labels):   # 확률에서 얼마나 떨어져있는가?
        loss = F.cross_entropy(outputs, labels)
        return loss
        
    def _calculate_accuracy(self, outputs, labels):   # 0.5보다 크면 1, 아니면 0으로 labeling
        active_logits = torch.argmax(outputs, dim=1)
        active_labels = torch.argmax(labels, dim=1)
        accuracy = accuracy_score(active_logits, active_labels)
        return accuracy

    # def _f1_score(self, outputs, labels):
    #     print('_f1_score : ')

    #     active_logits = outputs.view(-1)
    #     active_labels = labels.view(-1)
    #     print(active_logits, active_logits)
    #     f1 = F1Score()
    #     f1 = f1(active_logits, active_logits)
    #     print(f1)
    #     return f1

    def _convert_ids_to_labels(self, outputs, slot_labels):
        _, y_hat = torch.max(outputs, dim=2)
        y_hat = y_hat.detach().cpu().numpy()
        slot_label_ids = slot_labels.detach().cpu().numpy()

        slot_label_map = {i: label for i, label in enumerate(self.slot_labels_type)}
        slot_gt_labels = [[] for _ in range(slot_label_ids.shape[0])]
        slot_pred_labels = [[] for _ in range(slot_label_ids.shape[0])]

        for i in range(slot_label_ids.shape[0]):
            for j in range(slot_label_ids.shape[1]):
                if slot_label_ids[i, j] != self.pad_token_id:
                    slot_gt_labels[i].append(slot_label_map[slot_label_ids[i][j]])
                    slot_pred_labels[i].append(slot_label_map[y_hat[i][j]])

        return slot_pred_labels, slot_gt_labels

In [459]:
class Config(BertConfig):
    def __init__(self) :
        super().__init__()
        self.task= 'kor_nextsentence_prediction_'
        self.log_path= data_path+'/logs'
        self.bert_model = 'snunlp/KR-Medium' #'klue/bert-base'   
        self.max_len= 256
        self.train_batch_size= 32
        self.eval_batch_size= 32
        self.dropout_rate= 0.1
        self.gpus= torch.cuda.device_count()
    
config = Config()

## 모델 학습

In [460]:
preprocessor = Preprocessor(config.max_len)

In [461]:
dataset = {}
dataset['train'] = CorpusDataset(train, preprocessor.get_input_features)
dataset['val'] = CorpusDataset(val, preprocessor.get_input_features)
dataset['test'] = CorpusDataset(test, preprocessor.get_input_features)

In [462]:
model = BERTNextSentenceModel(config, dataset).cuda()

init


Some weights of the model checkpoint at snunlp/KR-Medium were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [463]:
logger = TensorBoardLogger(
    save_dir=os.path.join(config.log_path, config.task), version=3, name=config.task
)

acc_checkpoint_callback = ModelCheckpoint(
    dirpath=data_path+'/sts/checkpoints/'+ config.task, 
    filename="{epoch}_{val_acc:2f}_{other_metric:.2f}",
    verbose=True,
    monitor='val_acc',
    mode='max',
    save_top_k=1,
    save_last=True)

lrmonitor_callback = LearningRateMonitor(logging_interval='step')

In [467]:
trainer = pl.Trainer(
    #gpus=config.gpus,
    callbacks=[acc_checkpoint_callback, lrmonitor_callback],
    logger=logger,
    max_epochs=2,
)

trainer.fit(model)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
C:\Users\yeonok\anaconda3\envs\kobert\lib\site-packages\pytorch_lightning\trainer\trainer.py:1780: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  category=PossibleUserWarning,
C:\Users\yeonok\anaconda3\envs\kobert\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name    | Type                          | Params
----------------------------------------------------------
0 | softmax | Softmax                       | 0     
1 | model   | BertForNextSentencePrediction | 101 M 
2 | cls     | BertOnlyNSPHead               | 1.5 K 
3 | dropout | Dropout             

Validation sanity check: 0it [00:00, ?it/s]

C:\Users\yeonok\anaconda3\envs\kobert\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:261: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Validation sanity check:  50%|█████     | 1/2 [00:10<00:10, 10.83s/it]

C:\Users\yeonok\anaconda3\envs\kobert\lib\site-packages\ipykernel_launcher.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


...

C:\Users\yeonok\anaconda3\envs\kobert\lib\site-packages\ipykernel_launcher.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 0:  33%|███▎      | 532/1622 [5:24:42<11:05:16, 36.62s/it, loss=0.313, v_num=3]

C:\Users\yeonok\anaconda3\envs\kobert\lib\site-packages\ipykernel_launcher.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 0:  33%|███▎      | 533/1622 [5:25:17<11:04:37, 36.62s/it, loss=0.313, v_num=3]

C:\Users\yeonok\anaconda3\envs\kobert\lib\site-packages\ipykernel_launcher.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


### 토크나이저 비교

Mecab  
    https://i-am-wendy.tistory.com/27  
    https://velog.io/@kjyggg/%ED%98%95%ED%83%9C%EC%86%8C-%EB%B6%84%EC%84%9D%EA%B8%B0-Mecab-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0-A-to-Z%EC%84%A4%EC%B9%98%EB%B6%80%ED%84%B0-%EB%8B%A8%EC%96%B4-%EC%9A%B0%EC%84%A0%EC%88%9C%EC%9C%84-%EB%93%B1%EB%A1%9D%EA%B9%8C%EC%A7%80

In [ ]:
#result[0]을 예를들어보자.
prompt = news[0][0]
next_sentence = news[0][1]

print(prompt)
print(next_sentence)

 전남 목포시는 최근 이목이 집중되고 있는 근대역사문화공간 재생활성화 사업을 근대문화재 보존과 활용이라는 당초 취지대로 차질없이 추진하겠다고 밝혔다. 이 사업은 목포 원도심인 유달‧만호동 일대에 산재해 있는 근대건축물 등 문화유산
을 보존하고 보수‧정비하는 사업으로 금년부터 향후 5년간 총 사업비 500억원이 투입된다. 목포시는 원도심 일대의 근대경관을 회복하고 거주민 생활여건 개선과 관광인프라 확충 등을 통해 이 지역을 전국적인 근대 문화유산의 보고로 만


In [ ]:
kb_tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
mk_tokenizer = AutoTokenizer.from_pretrained('monologg/kobert')
sk_tokenizer = AutoTokenizer.from_pretrained('snunlp/KR-Medium')
mc_tokenizer = MeCab.Tagger()

print(f'klue/bert-base(의미 단위 토큰화) => {len(kb_tokenizer.tokenize(prompt))}토큰')
print(kb_tokenizer.tokenize(prompt))
print()
print(f'monologg/kobert(띄어쓰기 단위 토큰화) => {len(mk_tokenizer.tokenize(prompt))}토큰')
print(mk_tokenizer.tokenize(prompt))
print()
print(f'snunlp/KR-Medium(의미 단위 토큰화) => {len(sk_tokenizer.tokenize(prompt))}토큰')
print(sk_tokenizer.tokenize(prompt))
print()
mc = mc_tokenizer.parse(prompt).split('\n')
print(f"mecab(의미 단위 토큰화) => {len(mc)}토큰")
print(mc)

#klue/bert-base의 토크나이저의 결과가 가장 좋아보임

klue/bert-base(의미 단위 토큰화) => 65토큰
['전남', '목포시', '##는', '최근', '이목', '##이', '집중', '##되', '##고', '있', '##는', '근대', '##역', '##사', '##문화', '##공간', '재생', '##활', '##성', '##화', '사업', '##을', '근대', '##문화재', '보존', '##과', '활용', '##이', '##라는', '당초', '취지', '##대로', '차질', '##없이', '추진', '##하', '##겠다', '##고', '밝혔', '##다', '.', '이', '사업', '##은', '목포', '원도', '##심', '##인', '유', '##달', '‧', '만호', '##동', '일대', '##에', '산재', '##해', '있', '##는', '근대', '##건', '##축', '##물', '등', '문화유산']

monologg/kobert(띄어쓰기 단위 토큰화) => 31토큰
['[UNK]', '[UNK]', '최근', '[UNK]', '[UNK]', '있는', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '.', '이', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '있는', '[UNK]', '등', '[UNK]']

snunlp/KR-Medium(의미 단위 토큰화) => 64토큰
['전남', '목포', '##시는', '최근', '이', '##목이', '집중', '##되고', '있는', '근대', '##역', '##사', '##문화', '##공', '##간', '재', '##생활', '##성화', '사업을', '근대', '##문화', '##재', '보존', '##과', '활용', '##이라는', '당초', '취', '##지', '##대로'

In [ ]:
model = TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt=True)
kb_tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

encoding = kb_tokenizer(prompt, next_sentence, return_tensors='tf', padding=True)
encoding
#input_ids, token_type_ids, attention_mask가 각각 (1, 131) shape로 되어있음
#131 = 128자의 토큰 + [CLS], [SEP], [SEP] 인 것 같다...?

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForNextSentencePrediction: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


{'input_ids': <tf.Tensor: shape=(1, 67), dtype=int32, numpy=
array([[    2,  4997, 31441,  2259,  3744, 11086,  2052,  4154,  2496,
         2088,  1513,  2259,  5496,  2331,  2063,  6977, 29107,  6951,
         2424,  2047,  2267,  3639,  2069,  5496, 19611,  6628,  2145,
         3987,  2052, 23548,  6051,  5811,  3770,  8436,  4087,  3835,
         2205, 18395,  2088,  3705,  2062,    18,  1504,  3639,  2073,
         9312, 17638,  2361,  2179,  1490,  2448,   122, 31241,  2328,
         6394,  2170, 12389,  2097,  1513,  2259,  5496,  2332,  2174,
         2266,   886, 11319,     3]])>, 'token_type_ids': <tf.Tensor: shape=(1, 67), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0]])>, 'attention_mask': <tf.Tensor: shape=(1, 67), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
#result[0]을 예를들어보자.
prompt = result[0][0]
next_sentence = result[0][1]
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

encoding
#input_ids, token_type_ids, attention_mask가 각각 (1, 131) shape로 되어있음
#131 = 128자의 글자 + 2(), 3(), + ...?

{'input_ids': <tf.Tensor: shape=(1, 131), dtype=int32, numpy=
array([[    2,  4997, 31441,  2259,  3744, 11086,  2052,  4154,  2496,
         2088,  1513,  2259,  5496,  2331,  2063,  6977, 29107,  6951,
         2424,  2047,  2267,  3639,  2069,  5496, 19611,  6628,  2145,
         3987,  2052, 23548,  6051,  5811,  3770,  8436,  4087,  3835,
         2205, 18395,  2088,  3705,  2062,    18,  1504,  3639,  2073,
         9312, 17638,  2361,  2179,  1490,  2448,   122, 31241,  2328,
         6394,  2170, 12389,  2097,  1513,  2259,  5496,  2332,  2174,
         2266,   886, 11319,     3,  1498,  6628, 19521,  4687,   122,
         5336,  2205,  2259,  3639,  6233, 17008,  3797,  4914,    25,
         2440,  2366,  1668,  7518,  4667,  2028,  2252,  2052,  5113,
         2897,  2062,    18, 31441,  2259, 17638,  2361,  6394,  2079,
         5496,  2382,  2133,  2069,  4585, 19521,  5884,  2114,  3799,
         2173,  2332,  4076,  2145,  4066,  2179,  5747,  7237,   886,
         2069, 

In [ ]:
#kobert의 tokenizer를 사용하면?
from kobert_transformers.utils import get_tokenizer

to = get_tokenizer()
prompt = result[0][0]
next_sentence = result[0][1]
encoding = to(prompt, next_sentence, return_tensors='tf')

encoding
#shape이 왜 (1, 142)일까..?

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


{'input_ids': <tf.Tensor: shape=(1, 142), dtype=int32, numpy=
array([[   2, 4018, 2068, 7728, 6705, 5760, 4525, 3647, 6217, 7096, 4389,
        5887, 3860, 1221, 5808, 6926, 6493, 6238, 5453, 3969, 6545, 6573,
        7941, 2610, 1221, 5808, 6238, 7191, 2355, 7264, 5468, 5146, 7103,
        1630, 4609, 5812, 4419, 6883, 4555, 7786, 2261,   54, 3647, 2609,
        7086, 2068, 7728, 3533, 5859, 6745, 7119, 3574, 5793,    0, 6150,
        7925, 5872, 3806, 6896, 2640, 7191, 7848, 3860, 1221, 5808, 5384,
        7463, 6241, 1815, 2132, 7063, 6516,    3,  517, 7088, 2355, 7264,
        7788, 2374,    0, 7227, 6441, 7794, 2609, 7078, 1235, 5717, 5036,
         611, 5713, 4512, 2609, 6441,  614, 6861, 7096, 4767, 5900,   54,
        2068, 7728, 6705, 5760, 3533, 5859, 6745, 3806, 7095, 1221, 5808,
        5424, 5474, 7088, 5155, 7788,  875, 6263, 2717, 6918,  843, 5468,
        1080, 7119, 7754, 5135, 1824, 4756, 3647, 4329, 7088, 4014, 7206,
        1221, 5808, 2132, 7063, 6516, 7095, 2358, 

In [ ]:
logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

logits

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 0.16655698, -0.5226557 ]], dtype=float32)>

In [ ]:
softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [1]


In [ ]:
train, test = train_test_split(result, test_size=0.1, random_state=2)

len(train), len(test)

(282, 32)

### Korpus 데이터로 학습이 잘되면, news_class9x13000 활용 예정

## 마무리
맞춤법 검사 : py-hanspell